#CloakHat Patch Generation Pipeline

##1: Conda Environment Setup

`conda create -n cloakhat python=3.10 -y` <br>
`conda activate cloakhat`

PyTorch with CUDA <br>
`conda install pytorch torchvision pytorch-cuda=11.8 -c pytorch -c nvidia -y`

PyTorch3D for differentiable rendering <br>
`pip install "git+https://github.com/facebookresearch/pytorch3d.git"`

Detection models <br>
`pip install ultralytics`

Pip stuff <br>
`pip install opencv-python-headless matplotlib tqdm tensorboard pyyaml trimesh`